## SVM Optimization on a GPU

This [notebook](https://colab.research.google.com/drive/1AptayjRoDITNLmyfCc0T7z_xKFBlg2l-) shows the optimization of a multi-class, linear support vector machine using a simulation based optimizer. Any simulation based optimizer could be used with the cuda kernel in this notebook. I used KernelML, my custom optimizer, in this example. The runtime for this script should be set to use the GPU: Runtime->Change runtime type.

The original SVM formulation can be found in this paper: [Vapnik 1992](https://www.svms.org/training/BOGV92.pdf). There have been advances to the robustness of the algorithm since then. Please see [Robust Classifier 2019 section 6.1](https://www.mit.edu/~dbertsim/papers/Machine%20Learning%20under%20a%20Modern%20Optimization%20Lens/Robust_Classification.pdf). The robust impementation looks very tedious to implement. If you are interested in implementing it, please consider emailing me as well as looking at KernelML’s documentation. Email: rohankotwani@gmail.com.

SVM are typically optimized using Langrage multiplers and quadratic programming. However, this optimization process might not be fast enough, and we want to utilize GPUs. :) We will optimize the SVM primal form with brute force methods. Actually, using a simulation based approach is not such a bad idea because the computational complexity of training an SVM is O(N³), where N is the number of data points.

![SVM Primal Form](https://user-images.githubusercontent.com/21232362/70610188-24ef5180-1bd1-11ea-8cad-853b13b26ed8.png)


The Iris dataset was used to test the feasibility of using a GPU for simulated based optimization. The dataset has 150 rows and 4 columns. It is a small dataset, but the optimization procedure seems to work. The optimizer's documentation can be found [here](https://github.com/freedomtowin/kernelml/blob/master/README.md) and the cpu parallel implementation of this algorithm can be found [here](https://github.com/freedomtowin/kernelml/blob/master/kernelml-support-vector-machine.py).

In [1]:
!apt-get install nvidia-cuda-toolkit
!pip3 install numba

import os
os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/lib/nvidia-cuda-toolkit/libdevice"
os.environ['NUMBAPRO_NVVM'] = "/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so"

from numba import cuda
import numpy as np
import time


Reading package lists... Done
Building dependency tree       
Reading state information... Done
nvidia-cuda-toolkit is already the newest version (9.1.85-3ubuntu1).
You might want to run 'apt --fix-broken install' to correct these.
The following packages have unmet dependencies:
 nvidia-cuda-toolkit : Depends: nvidia-cuda-dev (= 9.1.85-3ubuntu1) but it is not going to be installed
E: Unmet dependencies. Try 'apt --fix-broken install' with no packages (or specify a solution).


In [2]:
!cd .. ; find . -name libdevice

./usr/local/lib/python3.7/dist-packages/jaxlib/cuda/nvvm/libdevice
./usr/local/cuda-10.1/nvvm/libdevice
./usr/local/cuda-11.0/nvvm/libdevice
./usr/local/cuda-10.0/nvvm/libdevice
./usr/lib/nvidia-cuda-toolkit/libdevice
./usr/lib/cuda/nvvm/libdevice
find: ‘./proc/576/task/576/net’: Invalid argument
find: ‘./proc/576/net’: Invalid argument


In [3]:
!cd .. ; find . -name libnvvm.so

./usr/local/cuda-10.1/nvvm/lib64/libnvvm.so
./usr/local/cuda-11.0/nvvm/lib64/libnvvm.so
./usr/local/cuda-10.0/nvvm/lib64/libnvvm.so
find: ‘./proc/576/task/576/net’: Invalid argument
find: ‘./proc/576/net’: Invalid argument


In [4]:
import os
os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/lib/nvidia-cuda-toolkit/libdevice"
os.environ['NUMBAPRO_NVVM'] = "/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so"


In [5]:
#test
from numba import cuda
import numpy as np
import time

@cuda.jit
def hello(data):
    data[cuda.blockIdx.x, cuda.threadIdx.x] = cuda.blockIdx.x

numBlocks = 5
threadsPerBlock = 10

data = np.ones((numBlocks, threadsPerBlock), dtype=np.uint8)

hello[numBlocks, threadsPerBlock](data)

print(data)

[[0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1]
 [2 2 2 2 2 2 2 2 2 2]
 [3 3 3 3 3 3 3 3 3 3]
 [4 4 4 4 4 4 4 4 4 4]]


/usr/local/lib/python3.7/dist-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_NVVM=/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so.

For more information about alternatives visit: ('https://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))
/usr/local/lib/python3.7/dist-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_LIBDEVICE=/usr/lib/nvidia-cuda-toolkit/libdevice.

For more information about alternatives visit: ('https://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))


In [6]:
#install optimizer
!pip install kernelml --upgrade

Requirement already up-to-date: kernelml in /usr/local/lib/python3.7/dist-packages (3.41)


In [7]:
from sklearn import datasets, preprocessing
import kernelml
from numba import cuda
import numba
import numpy as np
import pandas as pd
import numpy

data = datasets.load_iris()
y = data.target
X = data.data
features = data.feature_names

print('dataset shape: ',X.shape)
print()


#the first column is the intercept
X = np.column_stack((np.ones(X.shape[0]),X))

ohe = preprocessing.OneHotEncoder()

y = ohe.fit_transform(y.reshape(-1,1)).toarray()

#transform the input for the SVM
y = 2*(y-0.5)


@cuda.jit('void(float64[:,:,:],float64[:,:], float64[:,:], float64[:,:,:])')
def svm_output(out,x,y,w):
    t,i,j = cuda.grid(3)
    if t<w.shape[2] and i<x.shape[0] and j<y.shape[1]:
        for k in range(x.shape[1]):
          out[i,j,t] = out[i,j,t] + x[i,k]*w[k,j,t]
        out[i,j,t] = 1-out[i,j,t]*y[i,j]
        if out[i,j,t]<0:
          out[i,j,t] = 0

@cuda.jit('void(float64[:],float64[:,:], float64[:,:], float64[:,:,:], float64[:,:,:])')
def svm_loss(resX,x,y,w,out):
    t = cuda.grid(1)    

    if t<w.shape[2]:
      
      #first column is the intercept
      #exclude the first row of the weight matrix since it will map to
      #the intercept
      for i in range(w.shape[0]):
        for j in range(w.shape[1]):
          if i>0:
            resX[t] = resX[t] + w[i,j,t]*w[i,j,t]

      for i in range(out.shape[0]):
        for j in range(out.shape[1]):
          tmp = 1-out[i,j,t]*y[i,j]
          resX[t] = resX[t] + out[i,j,t]

def map_losses(X,y,w_list,C):
  
    N = w_list.shape[1]
    w_list = np.ascontiguousarray(w_list.reshape((X.shape[1],y.shape[1],N)))

    block_size=(8,32,1)

    out = np.zeros((X.shape[0],y.shape[1],N))
    d_out = cuda.to_device(out)
    
    grid_size_i = np.int(np.ceil((N+block_size[0] - 1) / block_size[0]))
    grid_size_j = np.int(np.ceil((X.shape[0]+block_size[1] - 1) / block_size[1]))
    grid_size_k = np.int(np.ceil((y.shape[1]+block_size[2] - 1) / block_size[2]))

    svm_output[[grid_size_i,grid_size_j,grid_size_k], block_size](d_out,X,y,w_list)

    result = np.ascontiguousarray(np.zeros(N))
    d_result = cuda.to_device(result)

    numBlocks = 5
    threadsPerBlock = 16

    threadsPerBlock = np.int(np.ceil((N+numBlocks - 1) / numBlocks))

    svm_loss[threadsPerBlock, numBlocks](d_result,X,y,w_list,d_out)
    loss = d_result.copy_to_host()
    return loss



runs = 6
zscore = 1.
simulation_factor = 300
volatility = 10
cycles = 20
volume = 10

kml = kernelml.KernelML(
         prior_sampler_fcn=None,
         posterior_sampler_fcn=None,
         intermediate_sampler_fcn=None,
         mini_batch_sampler_fcn=None,
         parameter_transform_fcn=None,
         loss_calculation_fcn=map_losses,
         batch_size=None)

C = np.float64(1.0)
args_list = [C]

kml.optimize(X,y,
                                args=args_list,
                                number_of_parameters=15,
                                number_of_realizations=runs,
                                number_of_random_simulations = simulation_factor,
                                number_of_cycles=cycles,
                                update_volatility = volatility,
                                update_volume=volume,
                                convergence_z_score=zscore,
                                prior_uniform_low=-1,
                                prior_uniform_high=1,
                                print_feedback=True)

def svm(x,w,):
    out = x.dot(w)
    return out

ytrue = data.target

print('target:',ytrue)

w = kml.kmldata.best_weight_vector.reshape(5,3)

ypred = np.argmax(X.dot(w),axis=1)

print('predicted:',ypred)

print('accuracy:',np.sum(ypred==ytrue)/ytrue.shape[0])

dataset shape:  (150, 4)

realization 0 loss 158.16097349112712 time 2.520176649093628
realization 1 loss 133.2024091569742 time 2.4601633548736572
realization 2 loss 121.51214856567874 time 1.8104660511016846
realization 3 loss 119.420475668468 time 2.6366240978240967
realization 4 loss 120.59664300813047 time 1.370295524597168
realization 5 loss 116.70607338779753 time 2.687905788421631
target: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
predicted: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 2 1 1 1 2 1 1 1
 1 1 1 2 1 1 1 1 1 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2